In [34]:
# https://github.com/savarin/python_for_ml/blob/master/1-0_First_Cut.ipynb
# https://github.com/savarin/neural-networks/blob/master/1-0_Prelude.ipynb
# https://github.com/savarin/neural-networks/blob/master/1-1_Basic_NN-Titanic.ipynb

In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(42)
df = pd.read_csv('train.csv')

In [36]:
df_train = df.iloc[:890, :]
df_test_dummy = df.iloc[890:, :]
scaler = StandardScaler()
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S']

In [37]:
df_train = df_train.drop(['Name', 'Ticket', 'Cabin'], axis=1)
age_mean = df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(age_mean)
fare_mean = df_train['Fare'].mean()
df_train['Fare'] = df_train['Fare'].fillna(fare_mean)
df_train['Embarked'] = df_train['Embarked'].fillna('S')
df_train['Sex'] = df_train['Sex'].map({'female':0, 'male':1})
pd.get_dummies(df_train['Embarked'], prefix='Embarked').head(10)
df_train = pd.concat([df_train, pd.get_dummies(df_train['Embarked'], prefix='Embarked')], axis=1)
df_train = df_train.drop(['Embarked'], axis=1)
df_train_notarget = df_train.iloc[:, 2:]
df_train_notarget.head(10)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.000000,1,0,7.2500,0,0,1
1,1,0,38.000000,1,0,71.2833,1,0,0
2,3,0,26.000000,0,0,7.9250,0,0,1
3,1,0,35.000000,1,0,53.1000,0,0,1
4,3,1,35.000000,0,0,8.0500,0,0,1
5,3,1,29.695891,0,0,8.4583,0,1,0
6,1,1,54.000000,0,0,51.8625,0,0,1
7,3,1,2.000000,3,1,21.0750,0,0,1
8,3,0,27.000000,0,2,11.1333,0,0,1
9,2,0,14.000000,1,0,30.0708,1,0,0


In [38]:
# test data
df_test = pd.read_csv("test.csv")
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df_test['Age'] = df_test['Age'].fillna(age_mean)
df_test['Fare'] = df_test['Fare'].fillna(fare_mean)
df_test['Embarked'] = df_test['Embarked'].fillna('S')
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1})
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')], axis=1)
df_test = df_test.drop(['Embarked'], axis=1)
df_test.to_csv("submission.csv",index=False)
df_test_notarget = df_test.iloc[:, 1:]
df_test_notarget.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,34.5,0,0,7.8292,0,1,0
1,3,0,47.0,1,0,7.0000,0,0,1
2,2,1,62.0,0,0,9.6875,0,1,0
3,3,1,27.0,0,0,8.6625,0,0,1
4,3,0,22.0,1,1,12.2875,0,0,1
5,3,1,14.0,0,0,9.2250,0,0,1
6,3,0,30.0,0,0,7.6292,0,1,0
7,2,1,26.0,1,1,29.0000,0,0,1
8,3,0,18.0,0,0,7.2292,1,0,0
9,3,1,21.0,2,0,24.1500,0,0,1


In [39]:
# scaler
X_train = scaler.fit_transform(df_train_notarget[features].values)
X_test = scaler.fit_transform(df_test_notarget[features].values)

# onehot
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

df_train_notarget.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.000000,1,0,7.2500,0,0,1
1,1,0,38.000000,1,0,71.2833,1,0,0
2,3,0,26.000000,0,0,7.9250,0,0,1
3,1,0,35.000000,1,0,53.1000,0,0,1
4,3,1,35.000000,0,0,8.0500,0,0,1
5,3,1,29.695891,0,0,8.4583,0,1,0
6,1,1,54.000000,0,0,51.8625,0,0,1
7,3,1,2.000000,3,1,21.0750,0,0,1
8,3,0,27.000000,0,2,11.1333,0,0,1
9,2,0,14.000000,1,0,30.0708,1,0,0


In [40]:
# Keras MLP
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

start = time()

model = Sequential()
model.add(Dense(input_dim=9, output_dim=100))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(output_dim=100))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(output_dim=2))
model.add(Activation("softmax"))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.fit(X_train, y_train_onehot, nb_epoch = 10, batch_size = 20, verbose=1)

print(str(time() - start))

/Users/kznx/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=9, units=100)`
  
/Users/kznx/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100)`
  # This is added back by InteractiveShellApp.init_path()
/Users/kznx/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2)`
  
/Users/kznx/anaconda/envs/tensorflow/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
890/890 [==============================] - 0s 480us/step - loss: 0.6428 - acc: 0.6562
Epoch 2/10
890/890 [==============================] - 0s 72us/step - loss: 0.5712 - acc: 0.7607
Epoch 3/10
890/890 [==============================] - 0s 77us/step - loss: 0.5290 - acc: 0.7730
Epoch 4/10
890/890 [==============================] - 0s 75us/step - loss: 0.5031 - acc: 0.7831
Epoch 5/10
890/890 [==============================] - 0s 74us/step - loss: 0.4834 - acc: 0.7966
Epoch 6/10
890/890 [==============================] - 0s 83us/step - loss: 0.4712 - acc: 0.7899
Epoch 7/10
890/890 [==============================] - 0s 83us/step - loss: 0.4705 - acc: 0.7989
Epoch 8/10
890/890 [==============================] - 0s 83us/step - loss: 0.4563 - acc: 0.8056
Epoch 9/10
890/890 [==============================] - 0s 82us/step - loss: 0.4526 - acc: 0.8056
Epoch 10/10
890/890 [==============================] - 0s 82us/step - loss: 0.4551 - acc: 0.8045
1.5497701168060303


In [60]:
# submit
# https://qiita.com/teru855/items/02bd885179bd8e39ba43

y_prediction = np.round(model.predict_proba(X_test))
y_class = y_prediction[:,1]
y_class.astype(np.int)

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [61]:
# テスト値を再読み込み
df_out = pd.read_csv("test.csv")
df_out["Survived"] = y_class.astype(np.int)

# outputディレクトリに出力する
df_out[["PassengerId","Survived"]].to_csv("submission.csv",index=False)